In [8]:
import numpy as np
import pandas as pd
import bqplot
from bqplot import pyplot as plt
from bqplot import pyplot as bplt
import ipywidgets as widgets
import ipyvolume as ipv

import plotly
import plotly.graph_objects as go
import plotly.express as px

import time

from datetime import datetime
from dateutil.relativedelta import relativedelta
from numpy import array
from matplotlib.pyplot import *
from nose.tools import assert_equal
from sklearn.datasets import load_iris, load_wine
from sklearn.preprocessing import MinMaxScaler
from ipyvolume.widgets import quickvolshow

def version_to_int_list(version):
    return [int(s) for s in version.split('.')]

print('plotly.__version__:')
print(plotly.__version__)
print()

print('version_to_int_list(plotly.__version__):')
print(version_to_int_list(plotly.__version__))
print()

print('version_to_int_list(\'3.8.0\'):')
print(version_to_int_list('3.8.0'))
print()

plotly.__version__:
4.14.3

version_to_int_list(plotly.__version__):
[4, 14, 3]

version_to_int_list('3.8.0'):
[3, 8, 0]



In [2]:
assert version_to_int_list(plotly.__version__) >= version_to_int_list('3.8.0'), 'Sunburst plots require Plotly >= 3.8.0'

pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)

pd.options.mode.chained_assignment = None

In [4]:
# ======================================================================
# We'll now explain simple 2d animations with few examples. All of our 
# examples consist of below mentioned common steps:
#
#    Create bqplot Chart
#    Create ipywidgets Button
#    Create callback function for the button which will update chart data
#    Register callback with button using on_click()
#    Create UI combining button and bqplot figure using ipywidgets layout 
#    options.
# ======================================================================
plt.figure(1, title='Line Chart')
np.random.seed(0)
n = 200
x = np.linspace(0.0, 10.0, n)
y = np.cumsum(np.random.randn(n))
plt.plot(x, y)
plt.show()



In [5]:
# The following example does not work and nothing shown in the notebook
# after this code snippet:
x, y, z = np.random.random((3, 10000))
ipv.quickscatter(x, y, z, size=1, marker="sphere")
ipv.show()

# Same result as above for the following code:
x, y, z, u, v = ipv.examples.klein_bottle(draw=False)
ipv.figure()
m = ipv.plot_mesh(x, y, z, wireframe=False)
ipv.squarelim()
ipv.show()

# Only controls shown but no figures after running the following code:
ds = ipv.datasets.aquariusA2.fetch()
ipv.quickvolshow(ds.data, lighting=True)

In [7]:
# ======================================================================
# Example 1
#
# Our first example consists of simple animation based on a scatter plot
# in bqplot. We'll be creating a simple scatter plot consisting of 100 
# random points. We have used bqplot's pyplot API to create a scatter 
# chart of 100 random points below.
# ======================================================================

## Chart Creation Logic
colors= ["red", "green", "blue", "orangered", "tomato", "lawngreen", 
         "lime", "pink", "gray", "black"]

figure_1 = plt.figure(animation_duration=1000, 
                       title="Random Data Scatter Chart")
figure_1.layout.width="700px"
figure_1.layout.height="500px"

x = np.random.rand(100)
y = np.random.rand(100)
scatter_1 = plt.scatter(x,y)

plt.xlabel("X")
plt.ylabel("Y")

plt.xlim(0,1)
plt.ylim(0,1);

# And here, a simple ipywidgets button:
btn = widgets.Button(description="Start", icon="play")

# Below we have created a callback function which will be called each time
# a button is clicked. In the logic of the callback function, we are 
# looping 10 times, generating 100 new random points, and updating chart
# data with this new 100 points. We then stop for 1 second before 
# proceeding with the next iteration. This loop of 10 iterations with a 
# pause of 1 second between each iteration will give us simple animation.
#
# Note that we are updating scatter data using the hold_sync() context 
# of the scatter chart. The main reason for this is that it'll update 
# data at the same time synchronously.

## Callback to Update Chart
def update_scatter_chart(btn):
    for i in range(10):
        x = np.random.rand(100)
        y = np.random.rand(100)
        idx = np.random.choice(np.arange(len(colors)))
        with scatter_1.hold_sync():
            scatter_1.x = x
            scatter_1.y = y
            scatter_1.colors = [colors[idx]]
        time.sleep(1)

btn.on_click(update_scatter_chart)

# Below we have combined button and scatter chart figure into one UI 
# using ipywidgets VBox() layout which will layout widgets passed to it
# vertically. We can click the button and it'll start the animation by
# calling the callback function.

## UI Combining Button & Chart
widgets.VBox([btn, figure_1])

In [9]:
# ======================================================================
# Example 2
#
# Our second example consists of the animation of the bar chart. We have
# created a simple bar chart of 7 bars each having a different lengths.
# We have first created a simple bar chart using default heights and colors.
# ======================================================================

# We have then created a button widget and a callback function as usual.
# The callback function will again loop 15 iterations and each time, 
# it'll shuffle the heights of the bar chart and update new heights into
# the chart. It'll pause for 1 second between each iteration to give us
# simple animation.

## Chart Creation Logic
colors= ["red", "green", "blue", "orangered", "tomato", "lawngreen", "lime", "black"]
heights = [10, 20, 30, 40, 50, 60, 70]

fig = bplt.figure(animation_duration=1000,
                  title="Random Data Bar Chart",
                  fig_margin={'top':60, 'bottom':60, 'left':150, 'right':150},
                  background_style={'fill':"lightgray"})

bar = bplt.bar(x=[1,2,3,4,5],y=heights)

bplt.xlabel("X")
bplt.ylabel("Y")

bplt.ylim(0,75);

## Callback to Update Chart
def update_bar_chart(btn):
    for i in range(15):
        idx = np.random.choice(np.arange(len(colors)))
        np.random.shuffle(heights)
        with bar.hold_sync():
            bar.y = heights
            bar.colors = [colors[idx]]
        time.sleep(1)

btn = widgets.Button(description="Start Anime", icon="play")

btn.on_click(update_bar_chart)

widgets.VBox([fig, btn])




Example 3

Our third example of creating simple animation using bqplot consist of a line chart. We'll be loading the apple OHLC dataset for the same. The apple OHLC dataset is easily available from yahoo finance. We'll be plotting the first line chart with prices of the close price of the stock. We have then created a callback function that will loop 10 times and each time chooses between Open, Close, High, Low and Adj Close prices to update the line chart. The steps followed to create this animation are the same as previous examples.


In [11]:
apple_df = pd.read_csv("./AAPL.csv", index_col=0, parse_dates=True)
apple_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-09-14,114.720001,115.930000,112.800003,115.360001,114.622765,140150100
2020-09-15,118.330002,118.830002,113.610001,115.540001,114.801620,184642000
2020-09-16,115.230003,116.000000,112.040001,112.129997,111.413406,154679000
2020-09-17,109.720001,112.199997,108.709999,110.339996,109.634850,178011000
2020-09-18,110.400002,110.879997,106.089996,106.839996,106.157211,287104900


In [13]:
## Chart Creation Logic
cols = ["Open","High","Low","Close","Adj Close"]

colors= ["red", "green", "blue", "orangered", "tomato", "lawngreen", "lime", "black"]

fig = bplt.figure(animation_duration=1000, background_style={'fill':"lightgray"},
                  legend_location = "top-left",
                  title="Apple September,2020-September,2021 Price Data Animation ")

line = bplt.plot(x=apple_df.index,y=apple_df["Close"],
                 display_legend=True, labels=["Close"])

bplt.xlabel("Date")
bplt.xlabel("Price ($)")

#bplt.ylim(150, 330)

## Callback to Update Chart
def update_line_chart(btn):
    for i in range(10):
        time.sleep(1)
        idx1 = np.random.choice(np.arange(len(colors)))
        idx2 = np.random.choice(np.arange(len(cols)))
        line.y = apple_df[cols[idx2]]
        line.colors = [colors[idx1]]
        line.labels=[cols[idx2]]

btn = widgets.Button(description="Play", icon='play')

btn.on_click(update_line_chart)

widgets.VBox([btn, fig])